In [0]:
%matplotlib inline

import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
import random
from PIL import Image
import PIL
from tensorflow.python.framework import ops
import math
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm


# Use second GPU -- change if you want to use a first one
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
print(tf.__version__)

### Define number of train and test images

In [0]:
train_size = 704
#test_size = 3690

## Generate and save Train Data

In [0]:
path = '/content/drive/My Drive/Drive1(Own)/Arrow224/train/'
classes = os.listdir(path)
train_size = 704
file_size = 160
print(classes)
index = 0
counter = 0
X_train = np.zeros((train_size, int(file_size), int(file_size), 3), dtype = 'uint8')
Y_train = np.zeros((train_size), dtype = 'uint8')

for i in classes:

    print('class: ', i)
    files = os.listdir(str(path) + str(i))

    for k in files:
        img = Image.open(str(path) + str(i) + '/' + str(k))
        img.load
        img = img.resize((int(file_size), int(file_size)), Image.ANTIALIAS)
        npimg = np.asarray( img, dtype="uint8" )
        X_train[counter,:,:,:] = npimg
        Y_train[counter] = classes.index(i)
        counter += 1
    
      
print(counter)

np.save('/content/drive/My Drive/Drive1(Own)/Arrow224/X_train.npy', X_train)
np.save('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_train.npy', Y_train)


## Load and Normalize Train and Test Data

In [0]:
trainx = np.load('/content/drive/My Drive/Drive1(Own)/Arrow224/X_train.npy')
trainy = np.load('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_train.npy')

#testx = np.load('')
#testy = np.load('')

X_train = trainx/255
X_train = X_train.astype('float16')
X_train = np.resize(X_train, (train_size, 160, 160, 3))

#X_test = testx/255
#X_test = X_test.astype('float16')
#X_test = np.resize(X_test, (test_size, 64, 64, 3))

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
  
Y_train = convert_to_one_hot(trainy, 3).T  
#Y_test = convert_to_one_hot(testy, 8).T

print(X_train.shape, Y_train.shape)
#print(X_test.shape, Y_test.shape)

## Define Training Network

In [0]:
def net1(X):

    gen1 = tf.layers.conv2d(X, 64, 9, 1, padding = 'valid')
    A1 = tf.nn.relu(gen1)    
    P1 = tf.nn.max_pool(A1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
    
    gen2 = tf.layers.conv2d(P1, 64, 3, 1, padding = 'SAME')
    A2 = tf.nn.relu(gen2)    
    P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
    
    gen3 = tf.layers.conv2d(P2, 1, 5, 1, padding = 'valid')
    A3 = tf.nn.tanh(gen3)  
    P3 = tf.nn.max_pool(A3, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')    
    
    
    P_fl = tf.contrib.layers.flatten(P3)
    fc = tf.contrib.layers.fully_connected(P_fl, 3, activation_fn = None)
    
    return P3, fc

In [0]:
def net2(X):

    A1 = tf.nn.relu(X)    
    
    gen11 = tf.layers.conv2d(A1, 8, 1, 1, padding = 'SAME')

    gen21 = tf.layers.conv2d(A1, 8, 1, 1, padding = 'SAME')
    gen22 = tf.layers.conv2d(gen21, 16, 3, 1, padding = 'SAME')

    gen31 = tf.layers.conv2d(A1, 8, 1, 1, padding = 'SAME')
    gen32 = tf.layers.conv2d(gen31, 8, 3, 1, padding = 'SAME')
    gen33 = tf.layers.conv2d(gen32, 8, 3, 1, padding = 'SAME')

    l4 = tf.keras.layers.concatenate([gen11, gen22, gen33])
    l5 = tf.keras.layers.concatenate([l4, A1])
    
    A6 = tf.nn.relu(l5)    
    
    P_fl = tf.contrib.layers.flatten(A6)
    fc = tf.contrib.layers.fully_connected(P_fl, 3, activation_fn = None)
    
    return fc

## Training Model


In [0]:

learning_rate = 0.0003
num_epochs = 1000
print('X_train shape', X_train.shape)  
print('Y_train shape', Y_train.shape)  
#print('X_test shape', X_test.shape)  
#print('Y_test shape', Y_test.shape)
print('Learning rate:', learning_rate)

(m, n_H0, n_W0, n_C0) = X_train.shape             
n_y = Y_train.shape[1]                            
costs = []                                                            
t1 = 0
t2 = 0

X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0], name = 'X')
Y = tf.placeholder(tf.float32, [None, n_y], name = 'Y')

P3, Z3 = net2(X)
Z4 = net1(P3)

cost1 = tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y)
cost1 = tf.reduce_mean(cost1)

cost2 = tf.nn.softmax_cross_entropy_with_logits(logits = Z4, labels = Y)
cost2 = tf.reduce_mean(cost2)

cost = cost1 + cost2


optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)    
init = tf.global_variables_initializer()    
saver = tf.train.Saver()

with tf.Session() as sess:

    sess.run(init)

    for epoch in range(num_epochs):
        
        _ , temp_cost = sess.run([optimizer, cost], feed_dict = {X: X_train, Y: Y_train})
        
        if epoch % 5 == 0:
            costs.append(temp_cost)
            t2 = time.time()
            print ('Time:', round(t2-t1, 4), "Cost after epoch %i: %f" % (epoch, temp_cost))
            t1 = time.time()
          
    print('Saving Model...')
    saver.save(sess, '/content/drive/My Drive/Drive1(Own)/Arrow224/model/model.ckpt')
    print('Model Saved...')

    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    # Calculate the correct predictions
    predict_op = tf.argmax(Z3, 1)
    correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
    # Calculate accuracy on the test set
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('accuracy', accuracy)
    train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
    #test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
    print("Train Accuracy:", train_accuracy)
    #print("Test Accuracy:", test_accuracy)
                 

In [0]:
with tf.Session() as sess:
  saver.restore(sess, "/content/drive/My Drive/Drive1(Own)/Arrow224/model/model.ckpt")
  print("Model restored.")
  train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
  print(train_accuracy)